In [18]:
import os
import struct
import json

path1 = 'recordings/recording_1562461507/server.rbs'
path2 = 'recordings/recording_1562461507/client.rbs'

file = open(path1, 'rb')

In [19]:
class FBSReader(object):
    def __init__(self, path):
        self.file = open(path, 'rb')
        version = self.file.read(12)
        if version != b'FBS 001.000\n':
            raise InvalidFBSFileError('Unrecognized FBS version: {}'.format(version))
        
        timestamp = self.file.read(4)
        (self.start,) = struct.unpack('!I', timestamp)
        header = self.file.readline()
        pos = self.file.tell()
        self.file.seek(pos, os.SEEK_SET)

    def __iter__(self):
        return self

    def read_safe(self, size=None):
        """
        We currently close our fbs files by killing them, so sometimes they end
        up with bad data at the end. Close our reader if we expect `size` bytes
        and get fewer.

        This is a hack and should be removed when we cleanly close our
        connections in fbs_writer.

        https://github.com/boxware/jiminy-envs/issues/41
        """
        bytes = self.file.read(size)
        if len(bytes) != size:
            # We unexpectedly got to the end of the file
            self.close()
            raise StopIteration
        return bytes

    def next(self):
        return self.__next__()

    def __next__(self):
        length_str = self.read_safe(4)
        if length_str == '':
            # Indicates a file with no trailer
            self.close()
            raise StopIteration
        (length,) = struct.unpack('!I', length_str)

        if length == 0:
            # Reached the end
            self.close()
            raise StopIteration()

        data = self.read_safe(length)
        timestamp_str = self.read_safe(4)
        (timestamp,) = struct.unpack('!I', timestamp_str)

        return data, self.start + timestamp/1000.

    def close(self):
        self.file.close()


In [16]:
fbs = iter(FBSReader(path1))

In [17]:
fbs.start

63685

In [349]:
file = open(path1, 'rb')

In [350]:
fbs_version = file.read(12)
fbs_version

b'FBS 001.000\n'

In [351]:
random_v = file.read(4)
print(random_v)
struct.unpack('!i', random_v)

b'\x00\x00\xf8\xc5'


(63685,)

In [352]:
rfb_info = file.read(12)
print(rfb_info)

b'RFB 003.003\n'


In [353]:
sec_type = file.read(4)
print(sec_type)
struct.unpack('!i', sec_type)

b'\x00\x00\x00\x01'


(1,)

In [354]:
randomstuff = file.read(20)
print(randomstuff)
#struct.unpack('!IHHI', randomstuff)

b'\x04\x00\x03\x00 \x18\x00\x01\x00\xff\x00\xff\x00\xff\x10\x08\x00\x00\x00\x00'


In [355]:
desktopNameLength = file.read(4)
print(desktopNameLength)
(d_len, ) = struct.unpack('!i', desktopNameLength)
print(d_len)

b'\x00\x00\x00\x17'
23


In [356]:
desktopName = file.read(d_len)
print(desktopName)

b'TigerVNC (b4c627851041)'


In [357]:
length_str = file.read(4)

In [358]:
length_str

b'\x00\x00\x00\x03'

In [359]:
(length,) = struct.unpack('!I', length_str)

In [360]:
length

3

In [361]:
paddedSize = (length + 3) & 0x7FFFFFFC

In [362]:
data = file.read(paddedSize)

In [363]:
data

b'\x00\x01\x00\x01'

In [332]:
timestamp_str = file.read(4)

In [333]:
(timestamp,) = struct.unpack('!I', timestamp_str)

error: unpack requires a buffer of 4 bytes

In [324]:
timestamp

655376

In [334]:
timestamp_str

b''

In [163]:
name_l = struct.unpack('!iii', b'\x00\x00\x00\x01\x04\x00\x03\x00\x18\x00\x01\x00')

In [164]:
name_l

(1, 67109632, 402653440)

In [273]:
struct.unpack('!I', b'\x04\x00\x03\x00')

(67109632,)

In [272]:
randomstuff

b'\x04\x00\x03\x00 \x18\x00\x01\x00\xff\x00\xff\x00\xff\x10\x08\x00\x00\x00\x00'

In [348]:
paddedSize

4